In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

### Data analysis

### Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [2]:
df = pd.read_parquet('../data source/yellow_tripdata_2023-01.parquet')

In [3]:
len(df.columns)

19

### Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [5]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration'] = df['duration'].dt.total_seconds()/60

In [6]:
std_dev = np.std(df['duration'], ddof=1)
std_dev

42.594351241920904

### Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [7]:
len(df['duration'])

3066766

In [8]:
df['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [9]:
df_cleaned = df[df['duration'] >= 1][df['duration'] <= 60]

In [10]:
df_cleaned['duration'].describe()

count    3.009173e+06
mean     1.420486e+01
std      9.939386e+00
min      1.000000e+00
25%      7.216667e+00
50%      1.155000e+01
75%      1.818333e+01
max      6.000000e+01
Name: duration, dtype: float64

In [11]:
pct_left = (len(df_cleaned['duration'])*100)/len(df['duration'])
pct_left

98.1220282212598

### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

In [12]:
categorical = ['PULocationID', 'DOLocationID']

In [13]:
df_categorical = df_cleaned[categorical].astype(str)

In [14]:
df_categorical.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Data columns (total 2 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   PULocationID  object
 1   DOLocationID  object
dtypes: object(2)
memory usage: 68.9+ MB


In [15]:
dv = DictVectorizer()
train_dict = df_categorical.to_dict(orient='records')

### Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data
What's the RMSE on train?

In [16]:
target = 'duration'

In [17]:
X_train = dv.fit_transform(train_dict)
y_train = df_cleaned[target].values

In [18]:
model = LinearRegression()

In [19]:
model.fit(X_train, y_train)

LinearRegression()

In [20]:
y_pred = model.predict(X_train)

In [21]:
mse = mean_squared_error(y_train, y_pred, squared=False)
r2 = r2_score(y_train, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")

#sns.distplot(y_pred, label='prediction')
#sns.distplot(y_train, label='actual')

Mean Squared Error: 7.65
R² Score: 0.41


### Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [ ]:
df_val = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
df_val['duration'] = df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']
df_val['duration'] = df_val['duration'].dt.total_seconds()/60

In [ ]:
df_val_cleaned = df_val[df_val['duration'] >= 1][df_val['duration'] <= 60]

In [ ]:
df_val_cat = df_val_cleaned[categorical].astype(str)

In [ ]:
df_val_cat.info()

In [ ]:
val_dict = df_val_cat.to_dict(orient='records')    

In [ ]:
X_test = dv.transform(val_dict)
y_test = df_val_cleaned[target].values

In [ ]:
y_val_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_val_pred, squared=False)
r2 = r2_score(y_test, y_val_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R² Score: {r2:.2f}")